In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import pymongo
from selenium.webdriver.common.keys import Keys
from pyquery import PyQuery as pq
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
class Search:
    def __init__(self, url, keyword, driver, db):
        self.url = url
        self.keyword = keyword
        self.driver = driver
        self.db = db
        
    # 进入苏宁易购首页    
    def get_front_page(self):    
        self.driver.get(self.url)
    
    # 进入商品页面
    def get_commodity_page(self):
        # 输入的搜索框
        input = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#searchKeywords'))
        ) 
        # 搜索按钮
        submit = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#searchSubmit")))
        input.send_keys(self.keyword) 
        submit.click()
     
    # 获得总页面数量
    def get_page_num(self):
        target = self.driver.find_element_by_css_selector('#bottom_pager > div > span.page-more')
        self.driver.execute_script("arguments[0].scrollIntoView();", target)
        time.sleep(3)
        total = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#bottom_pager > div > span.page-more')))
        total_text = total.text
        # 正则处理
        pattern = re.compile('\S\S(\d+).*?')
        result = re.search(pattern, total_text)
        self.total = int(result.group(0)[1:])
        
    # 获取商品信息并且存储
    def get_content(self):
        try:
            # 选择展示框
            WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#product-list .item-wrap')))
            
            html = self.driver.page_source
            doc = pq(html)
            # 获得所有商品的数据
            items = doc('#product-list .item-wrap').items()
            for item in items:
                proucts = {
                    'image': item.find(".img-block")('a')('img').attr('src').strip().replace('//', ' '),
                    'price': item.find('.def-price').text().replace('\n', ' '),
                    'title': item.find('.title-selling-point').text().strip().replace('\n', ' '),
                    'comment_num': item.find('.info-evaluate')('a')('i').text(),
                    'shop': item.find('.store-stock').text(),
                }
                print(proucts)
                # 保存在products集合中
                try:
                    self.db['products'].insert_one(proucts)
                except:
                    continue
        except:
            return
    
    # 进行商品页面的翻页
    def to_next_page(self):
        for page in range(1, self.total+1):
            print('当前页:%d/%d'%(page,self.total))
            try:
                # 找到输入框
                time.sleep(1)
                inputs = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#bottomPage')))
                time.sleep(1)
                inputs.clear()
                # 找到确定按钮
                submit = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR, "#bottom_pager > div > a.page-more.ensure")))
                inputs.send_keys(page)
                submit.send_keys(Keys.ENTER)
                target = self.driver.find_element_by_css_selector('#bottom_pager > div > span.page-more')
                # 将页面下拉至底部
                self.driver.execute_script("arguments[0].scrollIntoView();", target)  
                time.sleep(3)
                self.get_content()
                # 进行判定
                WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR, '#bottom_pager > div > a.cur')), str(page))
            except:
                print("wrong page!")
                return
        self.driver.close()

In [3]:
if __name__ == '__main__':
    url = 'https://www.suning.com/'
    keyword =  'apple'
    driver = webdriver.Chrome(ChromeDriverManager().install())
    # 连接MongoDB
    client = pymongo.MongoClient('localhost')
    db = client['suning']
    try:
        db.drop_collection('products')
        db.create_collection('products')
    except:
        db.create_collection('products')
    # 开始爬取
    search = Search(url, keyword, driver, db)
    search.get_front_page()
    search.get_commodity_page()
    search.get_page_num()
    search.to_next_page()



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\柠檬\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
D:\Anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.
D:\Anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


当前页:1/50


D:\Anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:82: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


{'image': ' uimgproxy.suning.cn/uimg1/pcpv/pcpv/iwogh/2021/11/10/17/iwoghBannerPicture/Nmhd17ksHA1636538329807.png_200w_200h_4e_374w_374h_4e_400w_400h_4e', 'price': '¥1299.00到手价', 'title': 'vivo Y70t 6GB+128GB 碧海蓝 5G全网通 4500mah大电池 双引擎闪充 4800万后置 大电池拍照三摄全面屏5G手机', 'comment_num': '700+', 'shop': '苏宁自营'}
{'image': ' imgservice1.suning.cn/uimg1/b2c/image/phDiGTtNpTjuNmc6k5gQ8Q.jpg_400w_400h_4e', 'price': '¥5599.00', 'title': 'Apple iPhone 12 128G 白色 移动联通电信5G全网通手机 A14仿生芯片 5G速度 瓷晶面板防跌落 航空级铝金属边框超漂亮', 'comment_num': '26万+', 'shop': '苏宁自营'}
{'image': ' imgservice1.suning.cn/uimg1/b2c/image/V9keTtchfh-82pdfTuAPDg.png_400w_400h_4e', 'price': '¥8208.00', 'title': '苹果(Apple) iPhone 12 Pro Max 256GB 海蓝色 移动联通电信5G全网通手机 双卡双待 iphone12proMax A14仿生芯片，6.7英寸超视网膜XDR显示屏，激光雷达扫描仪，超瓷晶面板，现实力登场！', 'comment_num': '2500+', 'shop': '质点旗舰店'}
{'image': ' uimgproxy.suning.cn/uimg1/pcpv/pcpv/iwogh/2021/11/10/17/iwoghBannerPicture/x6vdYlADrm1636536166844.png_400w_400h_4e', 'price': '¥2299.00', 'title': 'iQOO Z5 12+256G 蓝色起源

{'image': ' imgservice5.suning.cn/uimg1/b2c/image/wFBlThO29Y9WM0QXVlY0Gg.png_400w_400h_4e', 'price': '¥4399.00', 'title': '苹果(Apple) iPhone 11 128GB 紫色 简配版 移动联通电信4G全网通手机 （不含电源适配器和耳机）双卡双待 iphone11 2020新款 6.1英寸高清屏，全新双摄系统，A13仿生芯片，双卡双待。', 'comment_num': '1.4万+', 'shop': '华科手机专营店'}
{'image': ' imgservice5.suning.cn/uimg1/b2c/image/ytTYIisyB10b7yLFwLxXIw.png_400w_400h_4e', 'price': '¥4298.00', 'title': '苹果(Apple) iPhone 11 128GB 白色 简配版 移动联通电信4G全网通手机 2020新款 6.1英寸高清屏，全新双摄系统，A13仿生芯片，双卡双待。', 'comment_num': '1.5万+', 'shop': '质点旗舰店'}
{'image': ' uimgproxy.suning.cn/uimg1/pcpv/pcpv/iwogh/2021/11/03/09/iwoghBannerPicture/aR1cYi67IL1635904374993.png_400w_400h_4e', 'price': '¥6749.00到手价', 'title': 'Apple iPhone 13 (A2634) 256GB 午夜色 支持移动联通电信5G 双卡双待手机 解锁超能力！超先进双摄系统，超强耐用性，超高速芯片，超劲续航大提升！', 'comment_num': '200+', 'shop': '苏宁服务酷族苏宁专营店'}
{'image': ' uimgproxy.suning.cn/uimg1/pcpv/pcpv/iwogh/2021/11/03/09/iwoghBannerPicture/KiJAqfxFLU1635904652642.png_400w_400h_4e', 'price': '¥5799.00', 'title': 'Apple 苹果 iPh

...

{'image': ' imgservice4.suning.cn/uimg1/b2c/image/uLU7WKO_DN8AG6IUJEeRFw.jpg_400w_400h_4e', 'price': '¥2959.00', 'title': 'Apple/苹果iPhone XR 移动联通电信 全网通4G 智能手机 已激活 裸机 64GB 黄色 此款为裸机，出均为单机头，无配件（无充电头耳机数据线等配件），包装是白色通用包装，请知悉！！', 'comment_num': '30+', 'shop': '苏宁国际GTD海外专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/TRYDFcg0gOZEJm5Gh_hVlA.jpg_400w_400h_4e', 'price': '¥2949.00', 'title': '[全新正品]Apple/苹果 iPhone11 移动联通电信4G 全网通智能手机 美版有锁已激活[裸机]64GB 黑色 此款为裸机，出均为单机头，无配件（无充电头耳机数据线等配件），包装是白色通用包装，请知悉！！', 'comment_num': '9', 'shop': '苏宁国际GTD海外专营店'}
{'image': ' imgservice5.suning.cn/uimg1/b2c/image/FEQGxUJC7n_bLPbs2-zq6w.png_400w_400h_4e', 'price': '¥6407.00', 'title': 'iPhone13 128GB 粉色+Apple MagSafe 磁吸无线充电器 iPhone13 128GB 粉色+Apple MagSafe 磁吸无线充电器', 'comment_num': '100+', 'shop': '质点旗舰店'}
{'image': ' imgservice5.suning.cn/uimg1/b2c/image/CGHhMFNI-mvoL--fQyBq4g.png_400w_400h_4e', 'price': '¥11798.00', 'title': 'Apple iPhone 13 Pro 512GB 远峰蓝色+ 碎屏险套餐 Apple iPhone 13 Pro 512GB 远峰蓝色+ 碎屏险套餐', 'comment

{'image': ' imgservice3.suning.cn/uimg1/b2c/image/eu4KIH9HpiT6KbGe-_csxQ.jpg_400w_400h_4e', 'price': '¥12489.00', 'title': '[国家宝藏礼盒版]iPhone 12 Pro Max 512G 金色 5G全网通手机 国家宝藏定制礼盒套装 国家宝藏 IP定制配件礼盒 创意源自千里江山图；', 'comment_num': '6900+', 'shop': '苏宁自营'}
{'image': ' imgservice1.suning.cn/uimg1/b2c/image/o2xSNk59P_cmjdZ8YgMoMA.jpg_400w_400h_4e', 'price': '¥14597.00', 'title': 'iPhone13promax 1TB 远峰蓝色+苹果原装20W快充头 iPhone13promax 1TB 远峰蓝色+苹果原装20W快充头', 'comment_num': '2500+', 'shop': '华科手机专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/BQgafGbM7XYP-F7I4DVYVA.jpg_400w_400h_4e', 'price': '¥8547.00', 'title': 'iPhone13 512GB 午夜色+苹果原装20W快充头 iPhone13 512GB 午夜色+苹果原装20W快充头', 'comment_num': '3000+', 'shop': '华科手机专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/5-5YGmPV9ByLV6hVQ6nryg.png_400w_400h_4e', 'price': '¥13037.00', 'title': 'iPhone13pro 512GB 石墨色+AirPods Pro 无线充耳机 iPhone13pro 512GB 石墨色+AirPods Pro 无线充耳机', 'comment_num': '60+', 'shop': '质点旗舰店'}
{'image': ' imgservice2.suning.cn/uimg1/b2c

{'image': ' imgservice2.suning.cn/uimg1/b2c/image/1B3I6WgF2NvwkD6JiBtH_w.png_400w_400h_4e', 'price': '¥5287.00', 'title': 'iPhone 12 128GB+品胜PD20W快充Type-c快充充电器套装 iPhone 12 128GB+品胜PD20W快充Type-c快充充电器套装', 'comment_num': '900+', 'shop': '华科手机专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/D0VftuZ1b4LH-Ff4iKygaQ.jpg_400w_400h_4e', 'price': '¥7617.00', 'title': 'iPhone13mini 256GB 午夜色+AirPods Pro 无线充耳机 iPhone13mini 256GB 午夜色+AirPods Pro 无线充耳机', 'comment_num': '300+', 'shop': '华科手机专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/TQRR2bCjj9A1tSpMM2lE9Q.jpg_400w_400h_4e', 'price': '¥4547.00', 'title': '苹果11 128GB 简配版+苹果原装20W快充头原封充电器套装 苹果11 128GB 简配版+苹果原装20W快充头原封充电器套装', 'comment_num': '3900+', 'shop': '华科手机专营店'}
{'image': ' imgservice3.suning.cn/uimg1/b2c/image/wjME22ntlWnX0YPtA0xzyA.jpg_400w_400h_4e', 'price': '¥4278.00', 'title': 'iPhone 12 mini 64GB 黑色 + 碎屏险套餐,碎屏无忧 iPhone 12 mini 64GB 黑色 + 碎屏险套餐，碎屏无忧', 'comment_num': '2100+', 'shop': '华科手机专营店'}
{'image': ' imgservice2.suning.c

{'image': ' imgservice2.suning.cn/uimg1/b2c/image/sdHNtKceRkULMuzNKMlQLA.png_400w_400h_4e', 'price': '¥6987.00', 'title': 'iPhone13mini 128GB 蓝色+AirPods Pro 无线充耳机 iPhone13mini 128GB 蓝色+AirPods Pro 无线充耳机', 'comment_num': '300+', 'shop': '华科手机专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/8CRYVWG4MuiRAj4hP9ohOg.png_400w_400h_4e', 'price': '¥9937.00', 'title': 'iPhone13 512GB 蓝色+AirPods Pro 无线充耳机 iPhone13 512GB 蓝色+AirPods Pro 无线充耳机', 'comment_num': '60+', 'shop': '质点旗舰店'}
{'image': ' imgservice5.suning.cn/uimg1/b2c/image/3of-UmsPk9DwUl2pztNRpA.png_400w_400h_4e', 'price': '¥6967.00', 'title': 'iPhone 12 256G + AirPods 2 蓝牙无线耳机套餐 iPhone 12 256G + AirPods 2 蓝牙无线耳机套餐', 'comment_num': '3500+', 'shop': '华科手机专营店'}
{'image': ' imgservice3.suning.cn/uimg1/b2c/image/-GuEII0A5vbSei-NGRVBhQ.png_400w_400h_4e', 'price': '¥9397.00', 'title': 'iPhone13pro 256GB 金色+苹果原装20W快充头 iPhone13pro 256GB 金色+苹果原装20W快充头', 'comment_num': '700+', 'shop': '质点旗舰店'}
{'image': ' imgservice3.suning.cn/uimg1/b2c/imag

{'image': ' imgservice3.suning.cn/uimg1/b2c/image/mHMvKTQ7vqoMjsICdDbAeg.png_400w_400h_4e', 'price': '¥12007.00', 'title': 'iPhone13promax 256GB 金色+AirPods Pro 无线充耳机 iPhone13promax 256GB 金色+AirPods Pro 无线充耳机', 'comment_num': '60+', 'shop': '质点旗舰店'}
{'image': ' imgservice1.suning.cn/uimg1/b2c/image/IePse0tFVzE7i08tlLKOpA.png_400w_400h_4e', 'price': '¥9937.00', 'title': 'iPhone13 512GB 蓝色+AirPods Pro 无线充耳机 iPhone13 512GB 蓝色+AirPods Pro 无线充耳机', 'comment_num': '300+', 'shop': '华科手机专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/qbjJsHSngySvxXgRkK8B8w.jpg_400w_400h_4e', 'price': '¥8088.00', 'title': 'iPhone 12 Pro Max 128G 海蓝色 + 碎屏险一年套餐,碎屏无忧 iPhone 12 Pro Max 128G 海蓝色 + 碎屏险一年套餐，碎屏无忧', 'comment_num': '300+', 'shop': '华科手机专营店'}
{'image': ' imgservice4.suning.cn/uimg1/b2c/image/FlgJgwXCICx_FXkdePwesA.jpg_400w_400h_4e', 'price': '¥11087.00', 'title': 'iPhone13promax 128GB 远峰蓝色+AirPods Pro 无线充耳机 iPhone13promax 128GB 远峰蓝色+AirPods Pro 无线充耳机', 'comment_num': '60+', 'shop': '质点旗舰店'}
{'image'